## Imports

In [ ]:
from pathlib import Path

import datamate
import pandas as pd
import requests
from tqdm.notebook import tqdm

## Name For This Data

In [ ]:
name = "school-performance-data/raw"

## Load Necessary Configration

In [ ]:
key = datamate.config.get_key_for_name(name=name)

path_data = datamate.config.get_path_data_for_key(key=key)
logger = datamate.logging.get_logger(key=key)

logger.info(f"{key=} loaded for {name=}")
logger.info(f"{path_data=}")

## Data Information

- Link to setup the download https://www.find-school-performance-data.service.gov.uk/download-data
- Publication timetable https://www.find-school-performance-data.service.gov.uk/publication-timetable
- Guidance https://www.gov.uk/government/collections/school-and-college-performance-measures

In [ ]:
def mkdir(path: Path):
    path.mkdir(parents=True, exist_ok=True)
    
folder_template = "{year_start:%Y}-{year_end:%Y}"

## Historical Data Pull

In [ ]:
# TODO - automate this
data_information = {
    pd.Interval(pd.Timestamp("1991"), pd.Timestamp("1994")): dict(
        data=dict(filters="KS5", file_format="xls"),
        meta=None,
    ),
    # No data for 1994 - 1995
    pd.Interval(pd.Timestamp("1994"), pd.Timestamp("1995")): dict(
        data=None,
        meta=None,
    ),
    pd.Interval(pd.Timestamp("1995"), pd.Timestamp("2001")): dict(
        data=dict(filters="KS2,KS5", file_format="csv"),
        meta=None,
    ),
    pd.Interval(pd.Timestamp("2001"), pd.Timestamp("2010")): dict(
        data=dict(filters="KS2,KS4,KS5", file_format="csv"),
        meta=None,
    ),
    pd.Interval(pd.Timestamp("2010"), pd.Timestamp("2011")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS5,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS5,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL"),
    ),
    pd.Interval(pd.Timestamp("2011"), pd.Timestamp("2012")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4UNDERLYING,KS5,KS5UNDERLYING,DESTINATION,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4UNDERLYING,KS5,KS5UNDERLYING,DESTINATION,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED"),
    ),
    pd.Interval(pd.Timestamp("2012"), pd.Timestamp("2013")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,DESTINATION,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,DESTINATION,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED"),
    ),
    pd.Interval(pd.Timestamp("2013"), pd.Timestamp("2014")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,DESTINATION,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,DESTINATION,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL"),
    ),
    pd.Interval(pd.Timestamp("2014"), pd.Timestamp("2015")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,KS5MINSTANDARDS,VAQUAL,KS5UNDERLYING,VASUBJ,DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,KS5MINSTANDARDS,VAQUAL,KS5UNDERLYING,VASUBJ,DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL"),
    ),
    pd.Interval(pd.Timestamp("2014"), pd.Timestamp("2015")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,KS5MINSTANDARDS,VAQUAL,KS5UNDERLYING,VASUBJ,DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,KS5MINSTANDARDS,VAQUAL,KS5UNDERLYING,VASUBJ,DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL"),
    ),
    pd.Interval(pd.Timestamp("2015"), pd.Timestamp("2016")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS4DESTINATION,KS5DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS4DESTINATION,KS5DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL"),
    ),
    pd.Interval(pd.Timestamp("2016"), pd.Timestamp("2017")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS4DESTINATION,KS5DESTINATION,PUPDESTPROV,OFSTEDSCHOOLS,OFSTEDFE,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS4DESTINATION,KS5DESTINATION,PUPDESTPROV,OFSTEDSCHOOLS,OFSTEDFE,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS"),
    ),
    pd.Interval(pd.Timestamp("2017"), pd.Timestamp("2018")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS5MATHSCI,KS4DESTINATION,KS5DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS,KS5MATS", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS5MATHSCI,KS4DESTINATION,KS5DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS,KS5MATS"),
    ),
    pd.Interval(pd.Timestamp("2017"), pd.Timestamp("2018")): dict(
        data=dict(filters="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS5MATHSCI,KS4DESTINATION,KS5DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS,KS5MATS", file_format="csv"),
        meta=dict(regions="SPINE,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS5MATHSCI,KS4DESTINATION,KS5DESTINATION,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS,KS5MATS"),
    ),
    pd.Interval(pd.Timestamp("2018"), pd.Timestamp("2019")): dict(
        data=dict(filters="GIAS,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS5MATHSCI,KS4DESTINATION,KS5DESTINATION,KS5DESTINATIONHE,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS,KS5MATS", file_format="csv"),
        meta=dict(regions="GIAS,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS5MATHSCI,KS4DESTINATION,KS5DESTINATION,KS5DESTINATIONHE,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS,KS5MATS"),
    ),
    pd.Interval(pd.Timestamp("2019"), pd.Timestamp("2020")): dict(
        data=dict(filters="GIAS,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS5MATHSCI,KS4DESTINATION,KS5DESTINATION,KS5DESTINATIONHE,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS,KS5MATS", file_format="csv"),
        meta=dict(regions="GIAS,KS2,KS4,KS4PROV,KS4UNDERLYING,KS5,VAQUAL,KS5UNDERLYING,VASUBJ,KS5APPR,KS5MATHSCI,KS4DESTINATION,KS5DESTINATION,KS5DESTINATIONHE,PUPDESTPROV,PUPILABSENCE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL,KS2MATS,KS4MATS,KS5MATS"),
    ),
    # No data for 2019 - 2020
    pd.Interval(pd.Timestamp("2019"), pd.Timestamp("2020")): dict(
        data=None,
        meta=None,
    ),
    pd.Interval(pd.Timestamp("2020"), pd.Timestamp("2021")): dict(
        data=dict(filters="GIAS,KS4SUBJECTENTRIES,KS4EBACCENTRIES,KS5SUBJECTENTRIES,KS4DESTINATION,KS5DESTINATION,KS5DESTINATIONHE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL", file_format="csv"),
        meta=dict(regions="GIAS,KS4SUBJECTENTRIES,KS4EBACCENTRIES,KS5SUBJECTENTRIES,KS4DESTINATION,KS5DESTINATION,KS5DESTINATIONHE,CENSUS,WORKFORCE,SPENDPERPUPIL,SPENDPERPUPILGROUPED,SPENDPERPUPILFULL"),
    ),
    pd.Interval(pd.Timestamp("2021"), pd.Timestamp("2022")): dict(
        data=dict(filters="GIAS,KS4,KS4PROV,KS4SUBJECTENTRIES,KS5,KS5SUBJECTENTRIES,KS4DESTINATION,KS5DESTINATION,KS5DESTINATIONHE,CENSUS,KS4MATS,KS5MATS", file_format="csv"),
        meta=dict(regions="GIAS,KS4,KS4PROV,KS4SUBJECTENTRIES,KS5,KS5SUBJECTENTRIES,KS4DESTINATION,KS5DESTINATION,KS5DESTINATIONHE,CENSUS,KS4MATS,KS5MATS"),
    ),
}

In [ ]:
df_info = (
    pd.DataFrame(data_information)
    .transpose()
    .rename_axis("interval", axis="index")
    .reset_index()
)

year_range = pd.interval_range(start=pd.Timestamp("1991"), end=pd.Timestamp("2022"), freq="YS")

df_range = (
    pd.DataFrame(dict(year_range=year_range))
    .assign(temp=1)
    .merge(
        df_info.assign(temp=1),
        on="temp",
    )
    [lambda x: x.apply(lambda row: row["interval"].overlaps(row["year_range"]), axis=1)]
    .drop(columns="temp")
    .reset_index(drop=True)
    .assign(
        year_start=lambda x: x["year_range"].map(lambda y: y.left),
        year_end=lambda x: x["year_range"].map(lambda y: y.right),
        data=lambda x: x.apply(lambda row: None if row["data"] is None else dict(year_start=row["year_start"], year_end=row["year_end"], **row["data"]), axis=1),
        meta=lambda x: x.apply(lambda row: None if row["meta"] is None else dict(year_start=row["year_start"], year_end=row["year_end"], **row["meta"]), axis=1),
    )
)

In [ ]:
url_template = "https://www.find-school-performance-data.service.gov.uk/download-data?download=true&regions=0&filters={filters}&fileformat={file_format}&year={year_start:%Y}-{year_end:%Y}&meta=false"
url_meta_template = "https://www.find-school-performance-data.service.gov.uk/download-data?download=true&regions={regions}&filters=meta&fileformat=csv&year={year_start:%Y}-{year_end:%Y}&meta=true"

data_file_name_template = "{year_start:%Y}-{year_end:%Y}-data.zip"
meta_file_name_template = "{year_start:%Y}-{year_end:%Y}-meta.zip"

logger.info(f"{url_template=}")
logger.info(f"{url_meta_template=}")

In [ ]:
for _, row  in tqdm(df_range.iterrows()):
    year_start = row["year_start"]
    year_end = row["year_end"]
    
    path_folder = path_data.joinpath(folder_template.format(
        year_start=year_start, 
        year_end=year_end,
    ))
    mkdir(path=path_folder)

    # Data
    if row["data"] is not None:
        url = url_template.format(**row["data"])
        logger.info(f"Reading data {url=}")
        response = requests.get(url)
        logger.info(f"{response}")

        path_file = path_folder.joinpath(data_file_name_template.format(
            year_start=year_start,
            year_end=year_end,
        ))
        logger.info(f"Dumping data to {path_file=}")
        path_file.write_bytes(response.content)
    else:
        logger.info(f"No data available for {year_start=:%Y}, {year_end=:%Y}")
        
    # Meta
    if row["meta"] is not None:
        url = url_meta_template.format(**row["meta"])
        logger.info(f"Reading meta {url=}")
        response = requests.get(url)
        logger.info(f"{response}")

        path_file = path_folder.joinpath(meta_file_name_template.format(
            year_start=year_start,
            year_end=year_end,
        ))
        logger.info(f"Dumping meta to {path_file=}")
        path_file.write_bytes(response.content)
    else:
        logger.info(f"No meta available for {year_start=:%Y}, {year_end=:%Y}")